<a href="https://colab.research.google.com/github/kiraneranki/Autism-Spectral-Disorder/blob/main/Reinforcement_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Implementation of Reinforcement Learning using OpenAI Gym resource.**

In [ ]:
!pip install cmake gym[atari] scipy

**Create a environment for implementing "SmartCab Pickup and Dropoff Problem" using Reinforcement Learning Approach**

In [ ]:
import gym
env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



**The core gym interface is env, which is the unified environment interface. The following are the env methods that would be quite helpful to us:**
<li>
env.reset: Resets the environment and returns a random initial state.
<li>env.step(action): Step the environment by one timestep. Returns
observation: Observations of the environment
<li>reward: If your action was beneficial or not
done: Indicates if we have successfully picked up and dropped off a passenger, also called one episode
<li>info: Additional info such as performance and latency for debugging purposes
env.render: Renders one frame of the environment (helpful in visualizing the environment)
<li>
Note: We are using the .env on the end of make to avoid training stopping at 200 iterations, which is the default for the new version of Gym (reference).

**Important aspects of our problem:**

*There are 4 locations (labeled by different letters), and our job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions.*

Let's dive more into the environment.

In [ ]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


<b>
<li>The filled square represents the taxi, which is yellow without a passenger and green with a passenger.
<li>The pipe ("|") represents a wall which the taxi cannot cross.
<li>R, G, Y, B are the possible pickup and destination locations. 
<li>The blue letter represents the current passenger pick-up location, and the purple letter is the current destination.
</b>

As verified from above, we have an Action Space of size 6 and a State Space of size 500. As you'll see, our RL algorithm won't need any more information than these two things. All we need is a way to identify a state uniquely by assigning a unique number to every possible state, and RL learns to choose an action number from 0-5 where:
<b>
<li>0 = south
<li>1 = north
<li>2 = east
<li>3 = west
<li>4 = pickup
<li>5 = dropoff
</b>

<i>Recall that the 500 states correspond to a encoding of the taxi's location, the passenger's location, and the destination location.
Reinforcement Learning will learn a mapping of states to the optimal action to perform in that state by exploration, i.e. the agent explores the environment and takes actions based off rewards defined in the environment.
<b>The optimal action for each state is the action that has the highest cumulative long-term reward.</b></i>


In [ ]:
state = env.encode(2, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 228
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



**The Reward Table**

When the Taxi environment is created, there is an initial Reward table that's also created, called P. We can think of it like a matrix that has the number of *states as rows and number of actions as columns,* i.e. a $states \ \times \ actions$ matrix.
Since every state is in this matrix, we can see the default reward values assigned to our illustration's state:

In [ ]:
env.P[228]

{0: [(1.0, 328, -1, False)],
 1: [(1.0, 128, -1, False)],
 2: [(1.0, 248, -1, False)],
 3: [(1.0, 208, -1, False)],
 4: [(1.0, 228, -10, False)],
 5: [(1.0, 228, -10, False)]}

This dictionary has the structure ***{action: [(probability, nextstate, reward, done)]}.***

A few things to note:
<li>The 0-5 corresponds to the actions (south, north, east, west, pickup, dropoff) the taxi can perform at our current state in the illustration.

<li>In this env, probability is always 1.0.
The nextstate is the state we would be in if we take the action at this index of the dict
<li>All the *movement actions have a -1 reward* and the *pickup/dropoff actions have -10 reward* in this particular state. If we are in a state where the taxi has a passenger and is on top of the right destination, we would see a reward of 20 at the dropoff action (5)
<li>*done* is used to tell us when we have successfully dropped off a passenger in the right location. Each successfull *dropoff* is the end of an **episode**.</li>

Note that if our agent chose to explore action two (2) in this state it would be going East into a wall. The source code has made it impossible to actually move the taxi across a wall, **so if the taxi chooses that action, it will just keep acruing -1 penalties, which affects the long-term reward.**


In [ ]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 934
Penalties incurred: 342


In [29]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        #print(frame['frames'].getvalue())
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        env.s = state
        env.render()
print_frames(frames)

Timestep: 934
State: 0
Action: 5
Reward: 20
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


In [30]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [31]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns
from time import sleep
%matplotlib inline

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 1min 13s, sys: 17 s, total: 1min 30s
Wall time: 1min 15s


In [32]:
q_table[228]

array([ -2.363951  ,  -2.36394883,  -2.36395036,  -2.1220864 ,
       -11.27324682, -11.27320598])

In [33]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.94
Average penalties per episode: 0.0
